# PPA3 Database Table Viewer
If PPA is live and people are using it, and you want to view tables, please do so through this tool rather than Arc Pro. Viewing in Pro will create a schema lock on the tables, so when a user runs the tool, it will cause an error because it cannot log the run output to the locked database tables.

This viewer gets around this issue by creating an in-memory copy of desired gdb tables.

In [1]:
import pandas as pd

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

file_gdb = r'\\arcserver-svr\D\PPA3_SVR\PPA3_GIS_SVR\PPA3_run_data.gdb'
arcpy.env.workspace = file_gdb

fc_pmaster = 'project_master'
f_uid = 'project_uid'

df_pmaster = pd.DataFrame.spatial.from_featureclass(fc_pmaster)

def esri_object_to_df(in_esri_obj, esri_obj_fields, index_field=None):
    '''converts esri gdb table, feature class, feature layer, or SHP to pandas dataframe'''
    data_rows = []
    with arcpy.da.SearchCursor(in_esri_obj, esri_obj_fields) as cur:
        for row in cur:
            out_row = list(row)
            data_rows.append(out_row)

    out_df = pd.DataFrame(data_rows, index=index_field, columns=esri_obj_fields)
    return out_df

print([t for t in arcpy.ListTables()])


['rp_artexp_vmt', 'rp_artexp_econ', 'rp_artexp_eq', 'rp_artexp_mm', 'rp_artexp_sgr', 'rp_fwy_vmt', 'rp_fwy_cong', 'rp_fwy_mm', 'rp_fwy_econ', 'rp_fwy_frgt', 'rp_fwy_saf', 'rp_artsgr_sgr', 'cd_compactdev', 'cd_mixeduse', 'cd_houschoice', 'cd_naturpres', 'rp_artexp_cong', 'rp_artexp_frgt', 'rp_artexp_saf', 'cd_trnchoice', 'cd_existgasset', 'TEST_TABLE']


## View of most recent entries into master table

In [2]:
df_pmaster.sort_values(by='time_created', ascending=False).head()

,OBJECTID,project_uid,proj_name,juris,proj_type,aadt,pci,posted_speed,len_mi,comm_type,time_created,user_email,perf_outcomes,for_review,SHAPE
234,528,cc2867ef-29c0-4a33-bd1d-45a8c3552241,Yolo 80 ML S1,Yolo County,Freeway Expansion,156960.0,70.0,65.0,9.445633,Ag,2023-01-19 22:43:17.000000,sathish_prakash@dot.ca.gov,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13551879.4268, 4655654.535400003..."
233,527,e1976726-a851-4ee5-904f-80c0392fbe41,Yolo 80 ML Segment 1,Yolo County,Freeway Expansion,156960.0,70.0,65.0,9.454070,Ag,2023-01-19 22:33:39.000001,sathish_prakash@dot.ca.gov,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13551864.4184, 4655660.284500003..."
232,526,8c7c4c7d-654e-48f6-96f2-a077f1792a2b,FY2022-23 Road Overlay,County of Sutter,Arterial State of Good Repair,2004.0,60.0,50.0,17.181372,Ag,2023-01-19 14:16:18.000000,LCarrillo@co.sutter.ca.us,Reduce Congestion; Make a Safer Transportation...,NaN,"{""paths"": [[[-13529605.5491, 4716108.884599999..."
231,525,e7682f06-eb94-4d8b-8e28-044a5e76777b,Latrobe Road Interchange,El Dorado County,Arterial or Transit Expansion,30000.0,85.0,45.0,0.493907,Established Communities,2023-01-19 14:08:29.000000,ggammello@dewberry.com,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13477647.381099999, 4672642.0476..."
230,524,94f04d63-4c3e-4510-8d92-9e0bb948a618,US 50 at Empire Ranch Road Interchange,Folsom,Arterial or Transit Expansion,10000.0,75.0,45.0,0.520615,Developing,2023-01-19 12:52:25.000001,ggammello@dewberry.com,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13479968.579300001, 4671517.3962..."


## Find UID values that are in any of the data tables but not in the master table

In [3]:


def get_uids(in_tbl):
    uids = []

    try:
        with arcpy.da.SearchCursor(in_tbl, field_names=[f_uid]) as cur:
            for row in cur:
                uids.append(row[0])
    except:
        print(f"{in_tbl} does not have field {f_uid}. Skipping...")
        
    return uids

data_tables = [t for t in arcpy.ListTables()]
pmaster_uids = df_pmaster[f_uid].to_list()

out_dict = {}

fake_id = 'UID_NOT_FOUND' # placeholder ID used when running tool locally for testing

for t in data_tables:
    tbl_uids = get_uids(t)
    
    uids_not_in_master = []
    for uid in tbl_uids:
        if uid not in pmaster_uids and uid != fake_id:
            uids_not_in_master.append(uid)
            
    out_dict[t] = uids_not_in_master
    if len(out_dict[t]) > 0:
        print(f"table {t} has the follwing UIDs not in the project_master table: {out_dict[t]}")
        
total_missing = sum([len(v) for tbl, v in out_dict.items()])
if total_missing == 0: print("no IDs found in subreport tables that are not also in project_master")
    

TEST_TABLE does not have field project_uid. Skipping...
no IDs found in subreport tables that are not also in project_master


## Show specific subreport data and for specific projects

In [9]:

subrpt_tbl = 'rp_fwy_econ'
project_uid = None

fields = [f.name for f in arcpy.ListFields(subrpt_tbl)]

#uncomment if you want to filter which fields appear
# fields = [f_uid, 'crash_cnt', 'crash_100mvmt'] # [f.name for f in arcpy.ListFields(subrpt_tbl)]

df_master_fields = [f_uid, 'time_created', 'user_email', 'len_mi', 'aadt']
df_subrpt = esri_object_to_df(subrpt_tbl, esri_obj_fields=fields, index_field=None)


df_jn = df_pmaster[df_master_fields].merge(df_subrpt, on=f_uid, suffixes=('','_y')) \
    .sort_values(by='time_created', ascending=False)
df_jn.head()

,project_uid,time_created,user_email,len_mi,aadt,OBJECTID,acc_drive_alljob,acc_drive_edu
50,cc2867ef-29c0-4a33-bd1d-45a8c3552241,2023-01-19 22:43:17.000000,sathish_prakash@dot.ca.gov,9.445633,156960.0,94,526881.2500,479.817322
49,e1976726-a851-4ee5-904f-80c0392fbe41,2023-01-19 22:33:39.000001,sathish_prakash@dot.ca.gov,9.454070,156960.0,93,526881.2500,479.817322
48,1996cd29-85c1-4f2c-902f-f79542b31c51,2023-01-17 12:48:50.000000,nima.kabirinassab@dot.ca.gov,13.304194,181201.0,92,589030.1875,562.829773
47,fd2aa6db-d878-4f0e-8cb0-a63d1f4788a4,2023-01-17 12:43:38.000000,nima.kabirinassab@dot.ca.gov,13.052722,168670.0,91,583544.3750,559.334839
46,da426687-5e6e-41a7-a2d4-2e53eb876e4f,2023-01-17 12:14:23.000000,nima.kabirinassab@dot.ca.gov,3.749789,185355.0,89,712438.9375,727.313965


In [42]:
df_jn.loc[df_jn.crash_100mvmt > 100].head()

,project_uid,time_created,user_email,len_mi,aadt,crash_cnt,crash_100mvmt
21,55160fbd-9617-4c09-a05e-da8819a2b71c,2022-12-06 13:47:37,jhong@sacog.org,3.670994,5.0,136,463089.843750
45,fbc41452-2ed7-44ce-b1d1-3563bad0f73c,2023-01-17 06:06:46,rbissegger@markthomas.com,1.861210,58600.0,36,130.566345
